# Ermine Tutorial 05
## Correct for static and dynamic errors

by Sebastian Malkusch (c) 2021  
malkusch@med.uni-frankfurt.de  
Data Science| Klinische Pharmakologie  
Institut für Klinische Pharmakologie  
Goethe - Universität  
Theodor-Stern-Kai 7  
60590 Frankfurt am Main

## Abstract
In this tutorial, we cover static errors due to Endesfelder et al. (https://pubmed.ncbi.nlm.nih.gov/24522395/) and dynamic errors due to Savin and Doyle (https://pubmed.ncbi.nlm.nih.gov/15533928/) that occur when measuring molecular jumps in single-particle tracking experiments.

## Prerequisite
Nothing

## Import libraries
You need to install Ermine prior to usage.  
Run  
python -m pip install pyErmine  
from the command line 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import ermine as em

## Define experimental parameters
* define mobility modes for the model: priorModel_components  
* define time interval between two measurements : $\tau$
* define the time interval of a single measurement: $\sigma$
* define degrees of freedom for translational movement: dof
* define apparent extected mean squared displacements: $\mu_\text{apparent}$

In [2]:
model_components = 3
tau = 0.02
sigma = tau 
dof = 4
mu_apparent = np.array([[1306.66666667], [3866.66666667], [9466.66666667]])

## Apparent diffusion coefficient
According to Einstein, the diffusion coefficient for an error-free measurement is calculated as follows: 

In [3]:
D_apparent = mu_apparent / (dof * tau)
D_apparent

array([[ 16333.33333337],
       [ 48333.33333337],
       [118333.33333338]])

## Static error
Let's assume, that the first mobile mode with $\mu_\text{apparent} = 1307 \mu m^{2}$ is immobile. In this case, the measured displacement results from the fact that the position of a fixed molecule was repeatedly measured with errors. The measured displacement in this case is therefore the static measurement error or, more precisely, the localization accuracy $\epsilon$. It can be accessed by analysing the apparent extected mean squared displacement measured for the immobile mode using NeNA (https://pubmed.ncbi.nlm.nih.gov/24522395/):

In [4]:
epsilon = em.postprocessing.static_error(apparent_msd_d0 = mu_apparent[0], dof = dof)
print(str("The localization precision by NeNa is %.2f nm." %(epsilon)))

The localization precision by NeNa is 18.07 nm.


## Dynamic error
The position of a molecule is determined by the measured photon distribution. The photons are measured over a period of time $\sigma$. Within this period of time, however, the molecule does not stand still, but moves. Accordingly, the determined position of a molecule is an averaged position of all positions that the molecule has taken over the period of $\sigma$. Dies führt zu einem zusätzlichen Fehler für den korrigiert werden muss (https://pubmed.ncbi.nlm.nih.gov/15533928/):

In [5]:
D_corrected = em.postprocessing.calculate_diffusion_coefficient(expected_value = mu_apparent,
                                                                tau = tau,
                                                                dof =dof ,
                                                                sigma = tau,
                                                                epsilon = epsilon)
D_corrected

array([[     0.],
       [ 48000.],
       [153000.]])

## Re-estimating apparent ecpected values for the MSD
Conversely, ermine also allows the determination of the expected apparent MSD values for a given diffuson coefficient:

In [6]:
mu_apparent_estimated = em.postprocessing.calculate_expectation_value(diff_coeff = D_corrected,
                                                                      tau = tau,
                                                                      dof = dof, 
                                                                      sigma = tau,
                                                                      epsilon = epsilon)
mu_apparent_estimated

array([[1306.66666667],
       [3866.66666667],
       [9466.66666667]])